In [1]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import pandas as pd
import pickle as pkl
from tqdm.auto import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

/root/conda_env/chr_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
### load data 
import os
tmp = []
list_dir = os.listdir('./wiki_data')
list_dir = sorted(list_dir)
for path in list_dir:
    if 'wiki_2023_index' in path or 'ipynb' in path or 'my' in path:
        continue
    print(path)
    x = pd.read_parquet(os.path.join('./wiki_data',path))
    x['file'] = path
    tmp.append(x)
doc = pd.concat(tmp, axis=0).reset_index(drop=True)

a.parquet
b.parquet
c.parquet
d.parquet
e.parquet
f.parquet
g.parquet
h.parquet
i.parquet
j.parquet


In [ ]:
doc['context'] = doc['title']+" "+doc['text']

In [4]:
from tqdm.auto import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

# nltk.download('punkt')
# nltk.download('stopwords')

def word_process(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalnum()]
    return words
    

corpus = list(doc['context'])

In [ ]:
import multiprocessing
def process(data,idx):
    res = []
    if idx == 0:
        for item in tqdm(data):
            res.append(word_process(item))
    else:
        for item in data:
            res.append(word_process(item))
    import pickle as pkl
    with open(f'./tmp/corpus_{idx}.pkl','wb') as f:
        pkl.dump(res, f)
job = 60
sub_size = len(corpus) // job
remain = len(corpus) % job
subs = [corpus[i:i+sub_size] for i in range(0, len(corpus)-remain, sub_size)]
if remain:
    subs[-1].extend(corpus[-remain:])
process = [multiprocessing.Process(target=process, args=(subs[i], i)) for i in range(job)]
for pro in process:
    pro.start()
for pro in process:
    pro.join()

In [2]:
tokenized_corpus = []
job = 60
for item in tqdm(range(job)):
    with open(f'./tmp/corpus_{item}.pkl','rb') as f:
        tokenized_corpus.extend(pkl.load(f))

100%|██████████| 60/60 [11:37<00:00, 11.62s/it]


In [3]:
sum([len(x) for x in tokenized_corpus])

3407835861

In [16]:
model = Word2Vec(tokenized_corpus, vector_size=384, sg=1, epochs=10, min_count=5, window=32, dtype=np.float32, workers=-1)

In [21]:
model.save('./save/w2v.bin')

In [4]:
model = Word2Vec.load("./save/w2v.bin")

In [ ]:
import faiss
import numpy as np
index = faiss.IndexFlatIP(384)
for item in tqdm(tokenized_corpus):
    vec = [model.wv[word] for word in item if word in model.wv]
    if len(vec) == 0:
        output = np.full([384],10000, dtype=np.float32)
    else:
        output = sum(vec) / len(vec)
    output = output[None, :]
    output = output.astype(np.float32)
    faiss.normalize_L2(output)
    index.add(output)

 74%|███████▎  | 4627172/6286775 [3:43:56<1:08:33, 403.50it/s] 

In [ ]:
faiss.write_index(index, './wiki_index/w2v_index.bin')